In [ ]:
pip install "nltk == 3.4.5"

In [ ]:
%matplotlib inline
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace

# we will use CUDA if it is available
USE_CUDA = torch.cuda.is_available()
DEVICE=torch.device('cuda:0') if USE_CUDA else 'cpu'
print("CUDA:", USE_CUDA)
print(DEVICE)
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.set_default_dtype(torch.float64)

### ***Data***

In [ ]:
data = 'Indiana Chest X-Ray'

In [ ]:
import numpy as np

xray_vector = np.load('/Download/Datatset'+ data + '/XRayFeatures.npy',allow_pickle=True)
xray_vector = xray_vector.item()
train_xray = np.load('/Download/Datatset+ data' + '/XRayFeatures.npy')
train_xray = [train_xray[i].item() for i in range(len(train_xray))]
train_captions = np.load('/Download/Datatset'+ data + '/train_captions.npy')
train_captions = [train_captions[i].item() for i in range(len(train_captions))]

test_xray = np.load('/Download/Datatset' + data + '/test_xray.npy')
test_xray = [test_xray[i].item() for i in range(len(test_xray))]
test_captions = np.load('/Download/Datatset' + data + '/test_captions.npy')
test_captions = [test_captions[i].item() for i in range(len(test_captions))]

len(train_videos),len(train_captions), len(test_videos),len(test_captions)

In [ ]:
import spacy
import torchtext
from torchtext.data import Field, BucketIterator, TabularDataset
en = spacy.load('en')
EN_TEXT = Field(init_token='<sos>',
           eos_token='<eos>',
           tokenize=lambda captions : [ [tok.text for tok in en.tokenizer(sentence)] for sentence in captions],
           batch_first = True)
EN_TEXT.build_vocab(EN_TEXT.tokenize(train_captions))
len(EN_TEXT.vocab.stoi)

In [ ]:
from collections import defaultdict

train_references = defaultdict(list)
for i in range(len(train_captions)):
  train_references[train_xray[i]].append(train_captions[i].split())

test_references = defaultdict(list)
for i in range(len(test_captions)):
  test_references[test_xray[i]].append(test_captions[i].split())
  
len(train_references), len(test_references)

In [ ]:
from torch.utils.data import Dataset
class SampleDataset(Dataset):
  def __init__(self,xrayID):
    self.samples = []
    for i in range(len(xrayID)):
      if xrayID[i] in xray_vector :
        self.samples.append(xrayID[i])
 
  def __len__(self):
      return len(self.samples)
 
  def __getitem__(self,idx):
      return((self.samples[idx],torch.as_tensor(xray_vector[self.samples[idx]],dtype=torch.float64)))

train_dataset = SampleDataset(list(train_references.keys()))
test_dataset = SampleDataset(list(test_references.keys()))

In [ ]:
from torch.utils.data import DataLoader
from tqdm import notebook
import torch
train_loader = DataLoader(train_dataset,batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset,batch_size=64, shuffle=True, num_workers=2)

### ***Utility Functions***

In [ ]:
import nltk
nltk.download('wordnet')
from nltk.translate.bleu_score import corpus_bleu

def get_sentences(pred):
  sentences = []
  for i in range(pred.shape[0]):
    sentence = []
    for j in range(pred.shape[1]):
      if pred[i,j] == EN_TEXT.vocab.stoi['<eos>'] :
        break
      sentence.append(EN_TEXT.vocab.itos[pred[i,j].item()])
    sentences.append(' '.join(sentence))
  return sentences

def evaluate(inp, max_length):
  enc_hidden = model.encoder.initialize_hidden_state(inp.shape[0])
  enc_output, hidden = model.encoder(inp, enc_hidden)  
  dec_input = torch.full(size = (inp.shape[0],1), fill_value = EN_TEXT.vocab.stoi['<sos>'],device=DEVICE)

  pred = torch.empty(inp.shape[0],max_length,dtype = torch.int64)

  for i in range(max_length):
    predictions, hidden = model.decoder(dec_input, hidden, enc_output)
    output = torch.argmax(predictions,dim = 1)
    dec_input = output.view(inp.shape[0],1)
    pred[:,i] = output.cpu()
    
  return get_sentences(pred)


def get_scores(data_loader, data_references):
  references = []
  candidates = []
  for batch_no, (v,inp) in notebook.tqdm_notebook(enumerate(data_loader)) :
    o = evaluate(inp.to(device = DEVICE),30)
    for i in range(inp.shape[0]):
      l = sum(len(s) for s in data_references[v[i]])//len(data_references[v[i]])
      candidates.append(o[i].split()[:l])
      references.append(data_references[v[i]])
  result = {}
  result['BLEU1'] = corpus_bleu(references, candidates, weights=(1.0, 0, 0, 0))
  result['BLEU2'] = corpus_bleu(references, candidates, weights=(0.5, 0.5, 0, 0)) 
  result['BLEU3'] = corpus_bleu(references, candidates, weights=(0.33, 0.33, 0.33, 0))
  result['BLEU4'] = corpus_bleu(references, candidates, weights=(0.25, 0.25, 0.25, 0.25))
  result['METEOR'] = calculate_meteor_score(references, candidates)
  return result

### ***Load Model***

In [ ]:
batch = 64
seq_len = 30
input_size = 4096
enc_dim = 1024
embedding_dim = 256
vocab_size = len(EN_TEXT.vocab.stoi)
dec_units = 2048

In [ ]:
def load_checkpoint(filepath, model):
    checkpoint = torch.load(filepath,map_location=torch.device(DEVICE))
    model.load_state_dict(checkpoint['state_dict'])
    # print(model.eval())

In [ ]:
import os
model_names = ['Base Model', 'Proposed Model']

for model_name in model_names :
  print(model_name)
  path = '/Download/Dataset' + model_name + '/' + data + '/'

  if model_name == 'Base Model':
    %run '/Download/Dataset/BaseModel.ipynb'
  elif model_name == 'Proposed Model' :
    %run '/Download/Dataset/ProposedModel.ipynb'

  model = Encoder_Decoder_Model(seq_len, input_size, enc_dim, embedding_dim, vocab_size, dec_units)
  if USE_CUDA :
    model = model.cuda()

  start_epoch = 0
  while os.path.exists(path + 'checkpoint'+str(start_epoch+1) +'.pth'):
    start_epoch += 1
  # start_epoch = 49

  if start_epoch > 0:
    load_checkpoint(path + 'checkpoint'+str(start_epoch)+'.pth', model)
    print('{} Loaded from {} Epoch\n'.format(model_name, start_epoch))
  
  train_result = get_scores(train_loader, train_references)
  test_result = get_scores(test_loader, test_references)
  print('Metric Scores of the {} on the {} Train Dataset : '.format(model_name, data))
  print(train_result)
  print()
  print('Metric Scores of the {} on the {} Test Dataset : '.format(model_name, data))
  print(test_result)
  print("\n\n")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
root = '/Download/Dataset'
data = 'Indiana Chest X-ray'

def plot_multiple(models):
    plt.figure(figsize=(16,8))
    for model in models :
        path = root + model + '/' + data + '/losses.npy'
        losses = np.load(path, mmap_mode="r")
        ls = np.linspace(1,len(losses)+1,len(losses))
        plt.plot(ls,losses,label = model)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.show()
plot_multiple(model_names)